# Load and prepare dataset

In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error as mae
from pandas import read_csv

In [2]:
# Reading the dataset
dataset = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'], na_values=['nan','?'])

In [3]:
# Number of null values per column

print("Number of null values per column....")
print('\n')
print(dataset.isna().sum())

Number of null values per column....


Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64


In [4]:
#Handling missing values using forward interpolation method
dataset.interpolate(method='linear',limit_direction='forward', axis=0, inplace=True)
print("Number of null values per column ",dataset.isna().sum())

Number of null values per column  Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64


In [5]:
dataset.shape

(2075259, 7)

# Saving the datset

In [6]:
dataset.to_csv('household_power_consumption.csv')

# Resampling the data

## By days

In [7]:
# load the new file
dataset = pd.read_csv('household_power_consumption.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# resample data to daily
daily_groups = dataset.resample('D')
daily_data = daily_groups.sum()
# summarize
print(daily_data.shape)
print(daily_data.head())
# save
daily_data.to_csv('household_power_consumption_days.csv')

(1442, 7)
            Global_active_power  Global_reactive_power    Voltage  \
datetime                                                            
2006-12-16             1209.176                 34.922   93552.53   
2006-12-17             3390.460                226.006  345725.32   
2006-12-18             2203.826                161.792  347373.64   
2006-12-19             1666.194                150.942  348479.01   
2006-12-20             2225.748                160.998  348923.61   

            Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
datetime                                                                      
2006-12-16            5180.8             0.0           546.0          4926.0  
2006-12-17           14398.6          2033.0          4187.0         13341.0  
2006-12-18            9247.2          1063.0          2621.0         14018.0  
2006-12-19            7094.0           839.0          7602.0          6197.0  
2006-12-20            9313.0    

## By weeks

In [49]:
# load the new file
dataset = pd.read_csv('household_power_consumption.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# resample data to daily
daily_groups = dataset.resample('W')
daily_data = daily_groups.sum()
# summarize
print(daily_data.shape)
print(daily_data.head())
# save
daily_data.to_csv('household_power_consumption_weeks.csv')

(207, 7)
            Global_active_power  Global_reactive_power     Voltage  \
datetime                                                             
2006-12-17             4599.636                260.928   439277.85   
2006-12-24            17477.618               1176.174  2433008.21   
2006-12-31            19749.552               1453.126  2438445.48   
2007-01-07            14961.068               1348.954  2428490.09   
2007-01-14            16179.547               1590.541  2421917.62   

            Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
datetime                                                                      
2006-12-17           19579.4          2033.0          4733.0         18267.0  
2006-12-24           73994.4         11190.0         21351.0         77447.0  
2006-12-31           83078.0         14312.0         22675.0         67272.0  
2007-01-07           63122.2          5857.0         17599.0         54193.0  
2007-01-14           68864

## By months

In [45]:
# load the new file
dataset = pd.read_csv('household_power_consumption.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# resample data to daily
daily_groups = dataset.resample('M')
daily_data = daily_groups.sum()
# summarize
print(daily_data.shape)
print(daily_data.head())
# save
daily_data.to_csv('household_power_consumption_months.csv')

(48, 7)
            Global_active_power  Global_reactive_power       Voltage  \
datetime                                                               
2006-12-31            41826.806               2890.228  5.310732e+06   
2007-01-31            69017.296               5922.929  1.075399e+07   
2007-02-28            56496.828               4581.798  9.697733e+06   
2007-03-31            58862.721               5122.412  1.073652e+07   
2007-04-30            36529.192               5221.383  1.032721e+07   

            Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  
datetime                                                                      
2006-12-31          176651.8         27535.0         48759.0        162986.0  
2007-01-31          292264.4         56433.0         79274.5        329610.5  
2007-02-28          238497.0         47584.0         64640.0        270308.0  
2007-03-31          248774.5         60769.0        104762.0        290361.0  
2007-04-30   

# Checking for stationarity

### For Global_active_power case

In [78]:
# for days case
dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])


In [83]:
# for weeks case
dataset = read_csv('household_power_consumption_weeks.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])


In [85]:
# for months case
dataset = read_csv('household_power_consumption_months.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])


In [86]:
np.mean(dataset['Global_active_power'])

47137.75875

In [73]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries, window = 12, cutoff = 0.01):

    #Determing rolling statistics
    rolmean = timeseries.rolling(window).mean()
    rolstd = timeseries.rolling(window).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC', maxlag = 20 )
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    pvalue = dftest[1]
    if pvalue < cutoff:
        print('p-value = %.4f. The series is likely stationary.' % pvalue)
    else:
        print('p-value = %.4f. The series is likely non-stationary.' % pvalue)
    
    print(dfoutput)

In [74]:
test_stationarity(dataset['Global_active_power'])

Results of Dickey-Fuller Test:
p-value = 0.0038. The series is likely stationary.
Test Statistic                  -3.719967
p-value                          0.003841
#Lags Used                       2.000000
Number of Observations Used    204.000000
Critical Value (1%)             -3.462818
Critical Value (5%)             -2.875815
Critical Value (10%)            -2.574379
dtype: float64


### For Global_active_power case after differencing

In [75]:
first_diff = dataset['Global_active_power'] - dataset['Global_active_power'].shift(1)
first_diff = first_diff.dropna(inplace = False)
test_stationarity(first_diff)

Results of Dickey-Fuller Test:
p-value = 0.0000. The series is likely stationary.
Test Statistic                -1.484073e+01
p-value                        1.838843e-27
#Lags Used                     1.000000e+00
Number of Observations Used    2.040000e+02
Critical Value (1%)           -3.462818e+00
Critical Value (5%)           -2.875815e+00
Critical Value (10%)          -2.574379e+00
dtype: float64


# ACF and PACF

### We use the ACF and PACF plots to determine (p,d,q) parameters for the Arima Model. 

#### 'p' represents the number of AR terms
#### 'd' represents the the Integrative part of the model, i.e. if the data is non-stationary then which difference to take
#### 'q' represents the number of MA term

#### From the ACF plot we can learn if or how many MA terms to add and from the PACF plot we can learn if or how many AR terms to add.


### For Global_active_power case

In [76]:
import statsmodels.api as sm

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(dataset['Global_active_power'], lags=50, ax=ax1) # 
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(dataset['Global_active_power'], lags=50, ax=ax2)# , lags=40

### For Global_active_power case after differencing

In [77]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(first_diff, lags=50, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(first_diff, lags=50, ax=ax2)


# Model training and prediction

## For days

In [87]:
# arima forecast
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

warnings.simplefilter('ignore', ConvergenceWarning)

# split a univariate dataset into train/test sets
def split_dataset(data):
# split into standard weeks
	train, test = data[1:-328], data[-328:-6]
	# restructure into windows of weekly data
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# evaluate a single model
def evaluate_model(model_func, train, test):
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = model_func(history)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	predictions = array(predictions)
	# evaluate predictions days for each week
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores, predictions

# convert windows of weekly multivariate data into a series of total power
def to_series(data):
	# extract just the total power from each week
	series = [week[:, 0] for week in data]
	# flatten into a single series
	series = array(series).flatten()
	return series

# arima forecast
def arima_forecast(history):
	# convert history into a univariate series
	series = to_series(history)
	# define the model
	model = ARIMA(series, order=(7,0,0))    
	# fit the model
	model_fit = model.fit()
	# make forecast
	yhat = model_fit.predict(len(series), len(series)+6)
	return yhat

# load the new file
#dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

# split into train and test
train, test = split_dataset(dataset.values)
# define the names and functions for the models we wish to evaluate
models = dict()
models['arima'] = arima_forecast
# evaluate each model
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
for name, func in models.items():
	# evaluate and get scores
	score, scores, predictions = evaluate_model(func, train, test)
	# summarize scores
	summarize_scores(name, score, scores)
	# plot scores
	pyplot.plot(days, scores, marker='o', label=name)
# show plot
pyplot.legend()
pyplot.show()

arima: [408.145] 399.7, 417.1, 428.9, 394.2, 407.4, 311.0, 479.8


In [88]:
#calculate RMSE
rmse = sqrt(mean_squared_error(test[:, :, 0].flatten(),predictions.flatten())) 
print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
mape = np.mean(np.abs((test[:, :, 0].flatten() - predictions.flatten()) /test[:, :, 0].flatten())) * 100
print(f'Mean absolute percentage error is: {mape}')
mae_error = mae(test[:, :, 0].flatten(),predictions.flatten())
print(f'Mean Absolute Error is: {mae_error} ')

Root mean square error (RMSE) is: 408.1446241449729 kilowatts
Mean absolute percentage error is: 24.301575909193563
Mean Absolute Error is: 312.8943078546951 


In [89]:
%matplotlib qt
pyplot.plot(np.linspace(1,322,322), test[:, :, 0].flatten(), marker='o', label='actual')
pyplot.plot(np.linspace(1,322,322), predictions.flatten(), marker='x', label='predicted')
pyplot.legend(['actual', 'predicted'])
plt.xlabel('Days')
plt.ylabel('Global_active_power (in Kilowatts)')
plt.title('Actual vs. Predicted for daily case')
pyplot.show()

## For weeks

In [51]:
# arima forecast
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

warnings.simplefilter('ignore', ConvergenceWarning)

# split a univariate dataset into train/test sets
def split_dataset(data):
# split into standard months
	train, test = data[3:159], data[159:]
	# restructure into windows of monthly data
	train = array(split(train, len(train)/4))
	test = array(split(test, len(test)/4))
	return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for week day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# evaluate a single model
def evaluate_model(model_func, train, test):
	# history is a list of monthly data
	history = [x for x in train]
	# walk-forward validation over each month
	predictions = list()
	for i in range(len(test)):
		# predict the month
		yhat_sequence = model_func(history)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next month
		history.append(test[i, :])
	predictions = array(predictions)
	# evaluate predictions days for each month
	score, scores = evaluate_forecasts(test[:, :], predictions)
	return score, scores, predictions

# convert windows of weekly multivariate data into a series of total power
def to_series(data):
	# extract just the total power from each month
	series = [month[:] for month in data]
	# flatten into a single series
	series = array(series).flatten()
	return series

# arima forecast
def arima_forecast(history):
	# convert history into a univariate series
	series = to_series(history)
	# define the model
	model = ARIMA(series, order=(4,0,4))    
	# fit the model
	model_fit = model.fit()
	# make forecast
    
	yhat = model_fit.predict(len(series), len(series)+3)
	return yhat

# load the new file

dataset = read_csv('household_power_consumption_weeks.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

dataset = dataset['Global_active_power']
# split into train and test
train, test = split_dataset(dataset.values)
# define the names and functions for the models we wish to evaluate
models = dict()
models['arima'] = arima_forecast
# evaluate each model
days = ['Week1', 'Week2', 'Week3', 'Week4']
for name, func in models.items():
	# evaluate and get scores
	score, scores, predictions = evaluate_model(func, train, test)
	# summarize scores
	summarize_scores(name, score, scores)
	# plot scores
	pyplot.plot(days, scores, marker='o', label=name)
# show plot
pyplot.legend()
pyplot.show()

arima: [2126.841] 2176.6, 2492.7, 1627.2, 2120.1


In [52]:
#calculate RMSE
rmse = sqrt(mean_squared_error(test[:, :].flatten(),predictions.flatten())) 
print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
mape = np.mean(np.abs((test[:, :].flatten() - predictions.flatten()) /test[:, :].flatten())) * 100
print(f'Mean absolute percentage error is: {mape}')
mae_error = mae(test[:, :].flatten(),predictions.flatten())
print(f'Mean Absolute Error is: {mae_error} ')

Root mean square error (RMSE) is: 2126.8405602353782 kilowatts
Mean absolute percentage error is: 20.0523456377624
Mean Absolute Error is: 1543.1376370421615 


In [53]:
%matplotlib qt
pyplot.plot(np.linspace(1,48,48), test[:, :].flatten(), marker='o', label='actual')
pyplot.plot(np.linspace(1,48,48), predictions.flatten(), marker='x', label='predicted')
pyplot.legend(['actual', 'predicted'])
plt.xlabel('Weeks')
plt.ylabel('Global_active_power (in Kilowatts)')
plt.title('Actual vs. Predicted for Weekly case')
pyplot.show()

## For months



In [54]:
# arima forecast
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

warnings.simplefilter('ignore', ConvergenceWarning)

# split a univariate dataset into train/test sets
def split_dataset(data):
# split into standard years
	train, test = data[:36], data[36:]
	# restructure into windows of yearly data
	train = array(split(train, len(train)/12))
	test = array(split(test, len(test)/12))
	return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each month
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# evaluate a single model
def evaluate_model(model_func, train, test):
	# history is a list of yearly data
	history = [x for x in train]
	# walk-forward validation over each year
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = model_func(history)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next year
		history.append(test[i, :])
	predictions = array(predictions)
	# evaluate predictions days for each year
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores, predictions

# convert windows of weekly multivariate data into a series of total power
def to_series(data):
	# extract just the total power from each year
	series = [year[:, 0] for year in data]
	# flatten into a single series
	series = array(series).flatten()
	return series

# arima forecast
def arima_forecast(history):
	# convert history into a univariate series
	series = to_series(history)
	# define the model
	model = ARIMA(series, order=(12,1,12))    
	# fit the model
	model_fit = model.fit()
	# make forecast
	yhat = model_fit.predict(len(series), len(series)+11)
	return yhat

# load the new file
#dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

dataset = read_csv('household_power_consumption_months.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])

# split into train and test
train, test = split_dataset(dataset.values)
# define the names and functions for the models we wish to evaluate
models = dict()
models['arima'] = arima_forecast
# evaluate each model
days = ['Jan', 'Feb', 'March', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep','Oct','Nov','Dec']
for name, func in models.items():
	# evaluate and get scores
	score, scores, predictions = evaluate_model(func, train, test)
	# summarize scores
	summarize_scores(name, score, scores)
	# plot scores
	pyplot.plot(days, scores, marker='o', label=name)
# show plot
pyplot.legend()
pyplot.show()

C:\Users\Sharva\Anaconda3\envs\alda2\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:868: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  ' zeros.' % warning_description)


arima: [8714.432] 8874.6, 10334.8, 7175.1, 4201.6, 6302.6, 16704.0, 13493.1, 2575.1, 9418.3, 5421.6, 1769.8, 5284.2


In [55]:
#calculate RMSE
rmse = sqrt(mean_squared_error(test[:, :, 0].flatten(),predictions.flatten())) 
print(f'Root mean square error (RMSE) is: {rmse} kilowatts')
mape = np.mean(np.abs((test[:, :, 0].flatten() - predictions.flatten()) /test[:, :, 0].flatten())) * 100
print(f'Mean absolute percentage error is: {mape}')
mae_error = mae(test[:, :, 0].flatten(),predictions.flatten())
print(f'Mean Absolute Error is: {mae_error} ')

Root mean square error (RMSE) is: 8714.431736240027 kilowatts
Mean absolute percentage error is: 16.860117513990104
Mean Absolute Error is: 7629.570202349165 


In [56]:
%matplotlib qt
pyplot.plot(np.linspace(1,12,12), test[:, :, 0].flatten(), marker='o', label='actual')
pyplot.plot(np.linspace(1,12,12), predictions.flatten(), marker='x', label='predicted')
pyplot.legend(['actual', 'predicted'])
plt.xlabel('Months')
plt.ylabel('Global_active_power (in Kilowatts)')
plt.title('Actual vs. Predicted for monthly case')
pyplot.show()